In [120]:
import json
import re
from datetime import datetime, timedelta
from collections import defaultdict


In [121]:
filename = './data/2022sp-UA_GA.json'

In [123]:
def course_id(c, section_code='000'):
    return f"{c['subjectCode']['code']}-{c['subjectCode']['school']} {c['deptCourseId']}-{section_code}"

In [124]:
def p(s):
    print(json.dumps(s, indent=2))

In [125]:
INVALID_LOCATIONS = ['No', 'TBA', 'Online']


def recursively_default_dict():
    return defaultdict(recursively_default_dict)


def parse_meeting(meeting):
    start = datetime.strptime(meeting['beginDate'], '%Y-%m-%d %H:%M:%S')
    end = start + timedelta(minutes=meeting['minutesDuration'])
    weekday = start.weekday()
    # return weekday, start.strftime('%Y-%m-%d %H:%M:%S'), end.strftime('%Y-%m-%d %H:%M:%S')
    return weekday, start.strftime('%H:%M'), end.strftime('%H:%M')

def parse_location(location):
    loc_list = re.split(r'room\:?\s*|rm[:\s]|,\s*|\s\(',
                        location,
                        flags=re.IGNORECASE)
    building = loc_list[0].split('-')[0].replace('Bldg:', '').strip()
    room = loc_list[-1].strip()
    return building, room


def add_to_schedule(schedule, location, meetings):
    building, room = parse_location(location)
    if meetings and not (building in INVALID_LOCATIONS):
        for meeting in meetings:
            weekday, start, end = parse_meeting(meeting)
            schedule[building][room][weekday] = [
                *schedule[building][room][weekday], (start, end)
            ] if schedule[building][room].get(weekday) else [(start, end)]


def create_schedule(data):
    schedule = recursively_default_dict()
    for course in data:
        for session in course['sections']:
            add_to_schedule(schedule, session['location'], session['meetings'])
            if not session.get('recitations'): continue
            for recitation in session['recitations']:
                add_to_schedule(schedule, recitation['location'], recitation['meetings'])
    return schedule

def get_time_slots(times, start="08:00", end="22:00"):
    sorted_times = sorted(times, key=lambda x: x[0])
    time_slots = []
    if start < sorted_times[0][0]:
        time_slots.append((start, sorted_times[0][0]))
    for i in range(len(sorted_times)-1):
        time_slots.append((sorted_times[i][1],
                           sorted_times[i+1][0]))
    if end > sorted_times[-1][1]:
        time_slots.append((sorted_times[-1][1], end))
    return time_slots


def main(start="08:00", end="22:00"):
    with open(filename) as data_file:
        data = json.load(data_file)
        schedule = create_schedule(data)
        availabilities = recursively_default_dict()
        for building, rooms in schedule.items():
            for room, week in rooms.items():
                for weekday in range(7):
                    if weekday in week:
                        time_slots = get_time_slots(week[weekday], start, end)
                    else:
                        time_slots = [(start, end)]
                    availabilities[building][room][weekday] = time_slots
        return schedule, availabilities


In [128]:
schedule, availabilities = main()
with open('schedule.json', 'w') as outfile:
    json.dump(schedule, outfile, indent=2)
with open('availabilities.json', 'w') as outfile:
    json.dump(availabilities, outfile, indent=2)